In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys,os,re
import pandas as pd
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [ ]:
import os
import sys

Spark_Path='/opt/cloudera/parcels/CDH-5.8.2-1.cdh5.8.2.p0.3/lib/spark'

spark_home=Spark_Path + '/'#spark-' + SPARK_VER  +'-bin-hadoop2.6/'
os.environ['SPARK_HOME']=spark_home
os.environ['HADOOP_CONF_DIR']='/etc/hadoop/conf/'

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.9-src.zip'))

In [ ]:
from pyspark import SparkConf
sconf=SparkConf()

master='local'
AppName="Hadoop users"
num_executors=1

sconf.set('spark.master',master)
sconf.set('spark.executor.instances',str(num_executors))#Number of executors
#sconf.set('spark.shuffle.service.enabled',True)
#sconf.set('spark.dynamicAllocation.enabled',True)
sconf.set('spark.executor.memory','1g')
sconf.set('spark.driver.memory','1g')
#sconf.set('spark.executor.cores','2') # number of cores on same worker
sconf.set('spark.app.name',AppName) #Application Name
sconf.set('spark.app.id',AppName)

from pyspark import SparkContext
sc = SparkContext(conf=sconf)

In [ ]:
from pyspark import HiveContext
sqlContext = HiveContext(sc)
#from pyspark import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window
!cat /home/kentt/shared/code/spark_in_jupyter/sparklogo.txt

In [2]:
# Read raw data
with open('/home/kentt/shared/data/hadoop_nasjon_messy.txt', 'r') as myfile:
    raw_data=myfile.read().splitlines()

print "Data stored as a textfile on disk:\n"
print raw_data

IOError: [Errno 2] No such file or directory: '/home/kentt/shared/data/hadoop_nasjon_messy.txt'

In [5]:
# Load to Spark
raw_data_spark=sc.parallelize(raw_data)
raw_data_spark.take(2)

['# Number of users with "Hadoop" in their profile',
 'Norway 273,(8),Telenor,(8),DNV GL,(5),Affecto,(5),Tata Consultancy Services,(4),Microsoft']

In [6]:
# Drop header
raw_data_spark1 = raw_data_spark.filter(lambda row: row[0][0] != '#')\
.map(lambda row: row.split(','))
raw_data_spark1.take(2)

[['Norway 273',
  '(8)',
  'Telenor',
  '(8)',
  'DNV GL',
  '(5)',
  'Affecto',
  '(5)',
  'Tata Consultancy Services',
  '(4)',
  'Microsoft'],
 ['Sweden 863',
  '(55)',
  'Ericsson',
  '(44)',
  'Spotify',
  '(28)',
  'Klarna',
  '(25)',
  'King',
  '(18)',
  'KTH Royal Institute of Technology']]

In [7]:
# Parse Norway 273 as (273), Norway 
raw_data_spark2 = raw_data_spark1\
.map(lambda row: [row[0].split(' ')[1],row[0].split(' ')[0]]+row[1:])

raw_data_spark2.take(2)

[['273',
  'Norway',
  '(8)',
  'Telenor',
  '(8)',
  'DNV GL',
  '(5)',
  'Affecto',
  '(5)',
  'Tata Consultancy Services',
  '(4)',
  'Microsoft'],
 ['863',
  'Sweden',
  '(55)',
  'Ericsson',
  '(44)',
  'Spotify',
  '(28)',
  'Klarna',
  '(25)',
  'King',
  '(18)',
  'KTH Royal Institute of Technology']]

In [8]:
# Remove unwanted chars and set dtypes
raw_data_spark3=raw_data_spark2\
.map(lambda row: [i.translate(None,"()") for i in row])\
.map(lambda row: [int(i) if i.isdigit() else i for i in row])

raw_data_spark3.take(2)

[[273,
  'Norway',
  8,
  'Telenor',
  8,
  'DNV GL',
  5,
  'Affecto',
  5,
  'Tata Consultancy Services',
  4,
  'Microsoft'],
 [863,
  'Sweden',
  55,
  'Ericsson',
  44,
  'Spotify',
  28,
  'Klarna',
  25,
  'King',
  18,
  'KTH Royal Institute of Technology']]

In [9]:
# Data is parsed. Create schema and dataframe
schema = StructType([\
    StructField("num_country", IntegerType(), True),
    StructField("country", StringType(), True),
    StructField("num_top1", IntegerType(), True),
    StructField("top1_company", StringType(), True),
    StructField("num_top2", IntegerType(), True),
    StructField("top2_company", StringType(), True),
    StructField("num_top3", IntegerType(), True),
    StructField("top3_company", StringType(), True),
    StructField("num_top4", IntegerType(), True),
    StructField("top4_company", StringType(), True),
    StructField("num_top5", IntegerType(), True),
    StructField("top5_company", StringType(), True)])
    
hadoop_users = sqlContext.createDataFrame(raw_data_spark3, schema)
hadoop_users.show()

+-----------+-------+--------+--------------------+--------+----------------+--------+--------------------+--------+--------------------+--------+--------------------+
|num_country|country|num_top1|        top1_company|num_top2|    top2_company|num_top3|        top3_company|num_top4|        top4_company|num_top5|        top5_company|
+-----------+-------+--------+--------------------+--------+----------------+--------+--------------------+--------+--------------------+--------+--------------------+
|        273| Norway|       8|             Telenor|       8|          DNV GL|       5|             Affecto|       5|Tata Consultancy ...|       4|           Microsoft|
|        863| Sweden|      55|            Ericsson|      44|         Spotify|      28|              Klarna|      25|                King|      18|KTH Royal Institu...|
|       6170| France|     137|           Capgemini|     131|Amadeus IT Group|     117|              Orange|      99|              Criteo|      87|        Sopra 

In [11]:
# Dataframe with no. citizens pr. country
no_citizens = pd.read_csv('/home/kentt/shared/data/hadoop_nasjon_no_comment.csv',sep='\t')
no_citizens=sqlContext.createDataFrame(no_citizens[['no_country','no_citizens']])
no_citizens.show()

+----------+-----------+
|no_country|no_citizens|
+----------+-----------+
|    Sweden|      9.593|
|    Norway|      5.084|
|   Denmark|      5.614|
|   Finland|      5.439|
|        UK|       64.1|
|   Germany|      80.62|
|    Poland|      38.53|
|    France|      66.03|
+----------+-----------+



In [12]:
# Join if we want to
hadoop_users_with_no_citizens = hadoop_users.join(no_citizens, hadoop_users.country == no_citizens.no_country, 'inner').drop('no_country')

# Include column with normalized number of Hadoop users
hadoop_users_with_no_citizens_with_norm = hadoop_users_with_no_citizens\
.withColumn("num_country_norm",hadoop_users_with_no_citizens.num_country.cast("double") / hadoop_users_with_no_citizens.no_citizens) 


hadoop_users_with_no_citizens_with_norm.select("country","num_country","num_country_norm").orderBy("num_country",ascending=False).show()

+-------+-----------+------------------+
|country|num_country|  num_country_norm|
+-------+-----------+------------------+
|     UK|       9422|146.98907956318254|
| France|       6170| 93.44237467817659|
|Germany|       3703|45.931530637558915|
| Poland|       1368|35.504801453412924|
| Sweden|        863| 89.96143020952778|
|Finland|        621|114.17539988968561|
|Denmark|        419| 74.63484146775917|
| Norway|        273| 53.69787568843431|
+-------+-----------+------------------+



In [13]:
plot_results(hadoop_users_with_no_citizens_with_norm.toPandas(),'num_country')

In [14]:
plot_results(hadoop_users_with_no_citizens_with_norm.toPandas(),'num_country_norm')

In [ ]:
# Plot results
def plot_results(df,plot_col='num_country'):
    df["con_codes"]=['SWE','DEU','FRA','FIN','NOR','DNK','GBR','POL']
    data = [ dict(
            type = 'choropleth',
            locations = df['con_codes'],
            z = df[plot_col],
            text = df['top1_company']+':  '+df['num_top1'].astype(str)+'<br>'\
            +df['top2_company']+':  '+df['num_top2'].astype(str)+'<br>'\
            +df['top3_company']+':  '+df['num_top3'].astype(str)+'<br>'\
            +df['top4_company']+':  '+df['num_top4'].astype(str)+'<br>'\
            +df['top5_company']+':  '+df['num_top5'].astype(str)+'<br>',
            colorscale=[[0.0, 'rgb(165,0,38)'], [0.1111111111111111, 'rgb(215,48,39)']\
                        , [0.2222222222222222, 'rgb(244,109,67)'], [0.3333333333333333, 'rgb(253,174,97)']\
                        , [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)']\
                        , [0.6666666666666666, 'rgb(171,217,233)'], [0.7777777777777778, 'rgb(116,173,209)']\
                        , [0.8888888888888888, 'rgb(69,117,180)'], [1.0, 'rgb(49,54,149)']],
            autocolorscale = False,
            reversescale = True,
            locationmode = 'Europe',
            marker = dict(
                line = dict (
                    color = 'rgb(180,180,180)',
                    width = 0.5
                ) ),
            colorbar = dict(
                #autotick = False,
                tickprefix = '',
                title = 'No. of users'),
          ) ]

    layout = dict(
            title = '2016 - LinkedIn users with "Hadoop" in profile',
            geo = dict(
                scope='europe',
                projection=dict( type='merc' ),
                countrywidth='1',
                showcountries=True),
            width  = '1000',
            height = '800',
            margin = dict(
                l = '10',) 
            #    b = '1',)
         )


    fig = dict( data=data, layout=layout )
  
    return iplot(fig)